# Altair Themes & Multiple Charts

School of Information, University of Michigan

### Today's Plan:
1. Multiple Charts 
   
   a) Repeated Charts

   b) Faceted Charts
2. Altair Themes


### Resources & References 
* [Compound Charts](https://altair-viz.github.io/user_guide/compound_charts.html)
* [Repeat Charts Documentation](https://altair-viz.github.io/user_guide/generated/toplevel/altair.RepeatChart.html#altair.RepeatChart)
* [Facet Charts Documentation]()

In [1]:
# suppress warnings about future deprecations
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# imports we will use
import altair as alt
from vega_datasets import data


In [2]:
# load our data for today
cars = data.cars.url

## Repeated Charts

We often may want to look at multiple scatter plots or line plots to explore different dimensions of a large dataset and understand how the dimensions relate to eachother.

We can do this manually, using hconcat and vconcat.

In [3]:
# set the base chart
base = alt.Chart().mark_point().encode(
    color='Origin:N'
).properties(
    width=200,
    height=200
)

# define the chart object
chart = alt.vconcat(data=cars)
for y_encoding in ['Horsepower:Q','Acceleration:Q']: # the two y dimensions we want
    row = alt.hconcat()
    for x_encoding in ['Cylinders:Q','Miles_per_Gallon:Q']:
        # the x dimensions we want to look at
        row |= base.encode(x=x_encoding,y=y_encoding)
    chart &= row

chart

alt.VConcatChart(...)

This is a lot of code: we can create the same plot more efficiently using ```Chart.repeat()```.

The repeat method lets you specify a set of encodings for the row and/or column, which we can refer to in the chart's encoding with ```alt.repeat('row')``` and ```alt.repeat('column')```.

Let's look at the example below:

In [4]:
alt.Chart(cars).mark_point().encode(
    alt.X(alt.repeat("column"), type='quantitative'), # refers to alt.repeat('column')
    alt.Y(alt.repeat('row'),type='quantitative'), # refers to alt.repeat('row')
    color='Origin:N'
).properties(
    width=200,
    height=200
).repeat( # here's the Chart.repeat() object
    row=['Horsepower','Acceleration'],
    column=['Cylinders','Miles_per_Gallon']
)

alt.RepeatChart(...)

We can also use the ```Chart.repeat()``` to layer more efficiently. Below we see a layered chart of mean Miles per Gallon by Horsepower and mean Acceleration per Horsepower done manually, using alt.layer().

In [5]:
mpg = alt.Chart(cars).mark_line().encode(
    x='Horsepower:Q',
    y=alt.Y('Miles_per_Gallon:Q').aggregate('mean')
).properties(
    width=200,
    height=200
)

accel = alt.Chart(cars).mark_line(color='Orange').encode(
    x='Horsepower:Q',
    y=alt.Y('Acceleration:Q').aggregate('mean')
).properties(
    width=200,
    height=200
)

alt.layer(mpg,accel)

alt.LayerChart(...)

### <font color='#F7FF58'>Exercise 1: Can you recreate the above chart using alt.repeat?</font>

Hint: Try using the keyword 'layer'.

In [6]:
alt.Chart(cars).mark_line().encode(
    x='Horsepower:Q',
    y=alt.Y(alt.repeat('layer')).aggregate('mean'), # refers to alt.repeat('row')
    color = alt.ColorDatum(alt.repeat('layer'))
).properties(
    width=200,
    height=200
).repeat(
    layer=['Miles_per_Gallon','Acceleration']
)

alt.RepeatChart(...)

## Facet Charts

Similar to repeat charts, we might want to look at different subsets of data instead of different encodings. A chart where each panel represents a filtered view of the same data is also called a small multiple chart or grid chart.

We can create a chart like this manually, using hconcat.

In [7]:
# create the base chart
base = alt.Chart(cars).mark_errorband(extent="ci", borders=True).encode(
    x="year(Year)",
    y=alt.Y(
        "Miles_per_Gallon:Q",
        title="Miles per Gallon (95% CIs)",
    ),
).properties(width=160,
    height=160)

chart = alt.hconcat() # set the output chart
for ori in ['USA','Europe','Japan']: # iterate through the list of origin
    chart |=base.transform_filter(alt.datum.Origin==ori)
chart

alt.HConcatChart(...)

We can create the same chart more succinctly using ```.facet```.

In [8]:
alt.Chart(cars).mark_errorband(extent="ci", borders=True).encode(
    x="year(Year)",
    y=alt.Y(
        "Miles_per_Gallon:Q",
        title="Miles per Gallon (95% CIs)",
    ),
    color='Origin:N'
).properties(width=160,
    height=160
).facet( # use .facet and set column to be 'Origin:N'
        column='Origin:N',
    )

alt.FacetChart(...)

You can similarly set a 'row' argument in facet to change the orientation of the panels.

## Themes & Aesthetics

There are three ways to change how a plot looks in Altair:

1) **Global Config:** Changes an entire chart object
2) **Local Config:** Changes one mark (point, line, etc.) on a chart
3) **Encoding:** Encoding data (size, color, etc.) can impact chart properties.

Let's look at a basic chart:

In [9]:
# basic first chart
alt.Chart(cars).mark_point().encode(
    x='Acceleration:Q',
    y='Horsepower:Q'
)

alt.Chart(...)

We want to change the color of the marks on the **global level**, so that it impacts *an entire chart object*.

We can do this by using the .configure_* methods to change all of the aesthetics.

It's important to note that global config changes every mark in the chart and charts with top-level configuration (like .configure_mark) cannot be layered.

In [10]:
alt.Chart(cars).mark_point().encode(
    x='Acceleration:Q',
    y='Horsepower:Q'
).configure_mark( # this changes all of the marks on the chart
    opacity=0.2,
    color='red'
)

alt.Chart(...)

Now we want to change the color of the marks on the **local level**. To do this, we edit the local configuration setting of the mark_* method.

In our case, we can edit the ```color``` property in ```mark_point()```.

In [11]:
alt.Chart(cars).mark_point(opacity=0.2, color='red').encode(
    x='Acceleration:Q',
    y='Horsepower:Q'
)

alt.Chart(...)

### <font color='#F7FF58'>Exercise 2: What happens if you change both the local & global configuration?</font>

 Try changing the local config of the above chart to have blue marks with opacity of .4 and then change the global config to be pink marks with opacity of .1. Write up your conclusions.

In [12]:
alt.Chart(cars).mark_point(opacity=0.4, color='blue').encode(
    x='Acceleration:Q',
    y='Horsepower:Q'
).configure_mark( # this changes all of the marks on the chart
    opacity=0.1,
    color='pink'
)

alt.Chart(...)

*Insert written response here*

We can also set custom color schemes on the local level. Vega has pre-built color schemes that you can use: [Vega Color Scheme documentation](https://vega.github.io/vega/docs/schemes/).

Let's look at an example:

In [13]:
regular = alt.Chart(cars).mark_point().encode(
    x='Acceleration:Q',
    y='Horsepower:Q',
    color='Acceleration:Q'
)

scheme = alt.Chart(cars).mark_point().encode(
    x='Acceleration:Q',
    y='Horsepower:Q',
    color=alt.Color('Acceleration:Q',scale=alt.Scale(scheme='inferno'))
)

# Why do we do resolve scale? What happens when we remove this?
(regular | scheme).resolve_scale(color='independent')

alt.HConcatChart(...)

#### How do themes fit into this?

Altair maintains a registry of pre-made themes, that allow users to apply **global configuration** to all charts within a python session.

Altair's ```altair.theme``` module helps users interact with these premade themes.



In [14]:
# Let's look at all of registered themes.
# might be alt.theme.names() depending on your version
alt.themes.names()

['dark',
 'default',
 'excel',
 'fivethirtyeight',
 'ggplot2',
 'googlecharts',
 'latimes',
 'none',
 'opaque',
 'powerbi',
 'quartz',
 'urbaninstitute',
 'vox']

In [15]:
# Let's see what theme is currently active
# alt.theme.active
alt.themes.active

'default'

In [16]:
# Enable the theme 'quartz' and recreate our plot from above
# alt.theme.enable('quartz')
alt.themes.enable('quartz')
# print(alt.theme.active)
print(alt.themes.active)

# create chart
alt.Chart(cars).mark_point().encode(
    x='Acceleration:Q',
    y='Horsepower:Q'
)

quartz


alt.Chart(...)

We can see that this theme has changed the grid line colors, axis colors, and text colors. The size has also changed.

In [17]:
# We can change our chart to have no theme
# alt.theme.enable('none')
alt.themes.enable('none')


alt.Chart(cars).mark_point().encode(
    x='Acceleration:Q',
    y='Horsepower:Q'
)

alt.Chart(...)

In [18]:
# or we can return to default
# alt.theme.enable('default')
alt.themes.enable('default')

alt.Chart(cars).mark_point().encode(
    x='Acceleration:Q',
    y='Horsepower:Q'
)

alt.Chart(...)

### <font color='#F7FF58'>Exercise 3: Enable the 'dark' theme. What's different between this theme and 'none'?</font>

Hint: Try comparing chart.to_dict() when the theme 'none' is enabled versus the 'dark' theme.

In [19]:
alt.themes.enable('dark')

alt.Chart(cars).mark_point().encode(
    x='Acceleration:Q',
    y='Horsepower:Q'
)

alt.Chart(...)

*insert response here*

We can also make our own custom themes and register them locally.

A theme is a function that returns a dictionary of default values that will be added to the chart when rendered. Read more about themes on the [Altair User Guide](https://altair-viz.github.io/user_guide/customization.html#chart-themes).

Let's walk through the basic anatomy of a theme:

```
def theme_name():
return {
        "config": {
            "<TOP_LEVEL_OBJECT>": { # i.e. "title", "axisX", "legend", "range"
                "CONFIGURATION": "VALUE",
                "ANOTHER_ONE": "ANOTHER_VALUE",
                "MAYBE_A_SIZE": 14, # values can be a string, boolean, or number,
            },
            "<ANOTHER_OBJECT>": {
                "CONFIGURATION": "VALUE",
            }
        }
    }
```

The top-level objects can be component of the chart like the 'title', 'axisX', axisY' and general chart aspects like 'style', which changes attributes like text color for the axis labels and titles, and 'view', which changes the size of the chart.

Within the top-level object dictionary, you can set the 'font', 'fontSize, 'anchor' (alignment), 'fontColor', and other attributes. We will walk though an example of making our own theme below.


You can read the code for the built-in themes on the [vega themes github](https://github.com/vega/vega-themes/tree/master/src).


Let's try making our own theme.

In [20]:
# # Uncomment this for the newer version - 5.5.0
# # this registers the theme under the name 'blue' and enables
# @alt.theme.register("blue", enable=True)

# the theme is defined as a function
def blue(): #-> alt.theme.ThemeConfig: $ we want to update the current theme, so the output is ThemeConfig for version 5.5.0
    # we can set colors we frequently use to variables
    steel_blue = "#0582CA"
    sky_blue = "#00A6FB"
    return {
        "config": {
            "view": {"continuousWidth": 300, "continuousHeight": 300},
            "mark": {"color": steel_blue, "fill": sky_blue}, # you could just put color names, hex, rgb here
            "axis": {'grid': True,
                     'gridOpacity':.3,
                     'titleFontSize':18},
            'style': {'guide-title':{'fill': steel_blue}
                                     }
        }
    }

# Comment out the below lines for 5.5.0
alt.themes.register('blue',blue)
alt.themes.enable('blue')

ThemeRegistry.enable('blue')

In [21]:
# draw the chart
cars = data.cars.url
alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q'
)

alt.Chart(...)

### <font color='#F7FF58'>Exercise 4: Edit the code above to make your own theme called "spring".</font>

Set the grid color to be ``` "#FFB2E6" ``` (called "Lavender Pink" on [Coolors](https://coolors.co/ffffff-00072d-6320ee-ffb2e6-5c95ff)!), the axis labels to be "Electric Indigo" ``` "6320EE ```,
the mark fill and color to both be "Cornflower Blue" ``` "#5C95FF"```, the grid opacity to .5, and the axis label font size to be 16.

Recreate the same chart as above (Miles_per_Gallon by Horsepower) with the new theme.

In [22]:
def spring(): #-> alt.theme.ThemeConfig: $ we want to update the current theme, so the output is ThemeConfig for version 5.5.0
    # we can set colors we frequently use to variables
    Lavender_Pink = '#FFB2E6'
    Electric_Indigo = '#6320EE'
    Cornflower_Blue = '#5C95FF'
    return {
        "config": {
            "view": {"continuousWidth": 300, "continuousHeight": 300},
            "mark": {"color": Cornflower_Blue, "fill": Cornflower_Blue}, # you could just put color names, hex, rgb here
            "axis": {'grid': True,
                     'gridOpacity':.5,
                     'gridColor': Lavender_Pink,
                     'titleFontSize':16},
            'style': {'guide-title':{'fill': Electric_Indigo}
                                     }
        }
    }

# Comment out the below lines for 5.5.0
alt.themes.register('spring',spring)
alt.themes.enable('spring')

ThemeRegistry.enable('spring')

In [23]:
cars = data.cars.url
alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q'
)

alt.Chart(...)

### Color Palettes

We can also use the top-level argument of 'range' to set color palettes. There are multiple different color palettes you can set, for example:

* category: The default scheme for categorical data
* diverging: The default scheme for diverging quantitative data (e.g. correlation of -1 to 1)
* ordinal: The default scheme for ordinal data
* ramp: The default scheme for sequential quantitative ramps

You can read more about the different types of ranges you can define on the [Altair Range Config Documentation](https://vega.github.io/vega-lite/docs/scale.html#range-config).

Below we modified our blue theme to have a new default range palette.

In [24]:
# @alt.theme.register("blue2", enable=True)

def blue_2(): # -> alt.theme.ThemeConfig:
    steel_blue = "#0582CA"
    sky_blue = "#00A6FB"
    # define the desired palette as a list variable
    blue_purple_palette = ["#00A6FB",'#8963BA','#54428E']
    return {
        "config": {
            "view": {"continuousWidth": 300, "continuousHeight": 300},
            "mark": {"color": steel_blue, "fill": sky_blue},
            "axis": {'grid': True,
                     'gridOpacity':.3,
                     'titleFontSize':18},
            'style': {'guide-title':{'fill': steel_blue}
                                     },
            'range': {'ramp': blue_purple_palette} # set the sequential palette here
        }
    }

# Comment out the below lines for 5.5.0
alt.themes.register('blue_2',blue_2)
alt.themes.enable('blue_2')


alt.Chart(cars).mark_point(fill='none').encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Displacement:Q' # add the color encoding
)

alt.Chart(...)

### <font color='#F7FF58'>Exercise 5: Edit your "spring" theme to have a default categorical palette.</font>

You can choose any colors you'd like for the categorical palette. There are some fun tools like [Coolors](https://coolors.co/) that will help you generate aesthetically pleasing palettes.

In [25]:
def spring(): #-> alt.theme.ThemeConfig: $ we want to update the current theme, so the output is ThemeConfig for version 5.5.0
    # we can set colors we frequently use to variables
    Lavender_Pink = '#FFB2E6'
    Electric_Indigo = '#6320EE'
    Cornflower_Blue = '#5C95FF'
    return {
        "config": {
            "view": {"continuousWidth": 300, "continuousHeight": 300},
            "mark": {"color": Cornflower_Blue, "fill": Cornflower_Blue}, # you could just put color names, hex, rgb here
            "axis": {'grid': True,
                    'gridOpacity':.5,
                    'gridColor': Lavender_Pink,
                    'titleFontSize':16},
            'style': {'guide-title':{'fill': Electric_Indigo},
                    },
            'range': {'ramp': ['red','blue','green']} # set the sequential palette here
        }
    }

# Comment out the below lines for 5.5.0
alt.themes.register('spring',spring)
alt.themes.enable('spring')

ThemeRegistry.enable('spring')

### <font color='#F7FF58'>Exercise 6: Create a scatter plot 'Miles_per_Gallon' by 'Horsepower' and encode 'Origin' as the color.</font>

This should show off your new default categorical palette!

In [26]:
alt.Chart(cars).mark_point(fill='none').encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Displacement:Q' # add the color encoding
)

alt.Chart(...)

The themes are powerful tools for making reproducible visualizations with a consistent style. It is industry standard to follow a particular theme or style guide: as we see with FiveThirtyEight, LATimes, New York Times, and even the University of Michigan (check out [UofM's Design Resources](https://brand.umich.edu/design-resources/)).

### Do these theme config dictionaries look familiar?

Recall the week 7 lab, when we looked at the script with the chart spec in the HTML file that contained an altair chart.

```
<script>
    (function(vegaEmbed) {
      var spec = {"config": {"view": {"continuousWidth": 300, "continuousHeight": 300}}, "data": {"url": "https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/cars.json"}, "mark": {"type": "circle", "opacity": 0.5, "size": 80}, "encoding": {"color": {"field": "Origin", "type": "nominal"}, "x": {"field": "Horsepower", "type": "quantitative"}, "y": {"field": "Miles_per_Gallon", "type": "quantitative"}}, "params": [{"name": "param_50", "select": {"type": "point", "fields": ["cutoff"]}, "bind": {"input": "range", "max": 230.0, "min": 46.0, "name": "cutoff", "step": 1}, "value": 230.0}], "transform": [{"filter": "(datum.Horsepower < param_50.cutoff)"}], "$schema": "https://vega.github.io/schema/vega-lite/v5.8.0.json"};
      var embedOpt = {"mode": "vega-lite"};

      function showError(el, error){
          el.innerHTML = ('<div style="color:red;">'
                          + '<p>JavaScript Error: ' + error.message + '</p>'
                          + "<p>This usually means there's a typo in your chart specification. "
                          + "See the javascript console for the full traceback.</p>"
                          + '</div>');
          throw error;
      }
      const el = document.getElementById('vis');
      vegaEmbed("#vis", spec, embedOpt)
        .catch(error => showError(el, error));
    })(vegaEmbed);

  </script>
```

The chart config is one of the arguments in the var spec : **var spec = {"config": {"view": {"continuousWidth": 300, "continuousHeight": 300}}**. 

When we define the theme as above, we are changing this configuration on the global level.

### <font color='#F7FF58'>Exercise 7: What happens if you rerun all of the cells in your notebook? Does the first chart look how you expect?</font>

If not, how can you get the chart to look how it originally did **without changing any code**?

*insert response here*